In [1]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
lem = WordNetLemmatizer()
RANDOM_STATE = 50
UNK_ID = 1
PAD_ID = 0
MAX_LEN = 20

2022-07-28 01:12:50.511670: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-28 01:12:50.511699: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vaibhav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/vaibhav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vaibhav/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/vaibhav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def clean_string(s):    
    s =  re.sub(r'(?<=[^\s0-9])(?=[.,;?])', r' ', s)
    s = re.sub(r'\((\d+)\)', r'', s)
    s = re.sub(r'\s\s', ' ', s)
    s = re.sub(r"[^A-Za-z0-9(),!?\'`]", " ", s)
    s = re.sub(r"\'s", " \'s", s)
    s = re.sub(r"\'ve", " \'ve", s)
    s = re.sub(r"n\'t", " n\'t", s)
    s = re.sub(r"\'re", " \'re", s)
    s = re.sub(r"\'d", " \'d", s)
    s = re.sub(r"\'ll", " \'ll", s)
    s = re.sub(r",", " , ", s)
    s = re.sub(r"!", " ! ", s)
    s = re.sub(r"\"", " \" ", s)
    s = re.sub(r"\(", " ( ", s)
    s = re.sub(r"\)", " ) ", s)
    s = re.sub(r"\?", " ? ", s)
    s = re.sub(r"\s{2,}", " ", s)
    s = re.sub(r"\.", " . ", s)
    s = re.sub(r"., ", " , ", s)
    s = re.sub(r"\\n", " ", s)
    return s.strip().lower()

def create_train_valid(features,labels,train_fraction = 0.7,max_valid=500):

    features,labels = shuffle(features,labels,random_state = RANDOM_STATE)

    train_end = max(int(train_fraction*len(labels)),len(labels)-max_valid)

    train_features = np.asarray(features[:train_end])
    valid_features = np.asarray(features[train_end:])

    train_labels = np.asarray(labels[:train_end])
    valid_labels = np.asarray(labels[train_end:])
    
    return train_features,valid_features,train_labels,valid_labels

In [3]:
data = pd.read_csv('../datasets/extended_mustard_aug.csv')

features = data['text'].to_list()
labels = data['sarcasm'].to_list()

X_train, X_valid, y_train, y_valid = create_train_valid(features,labels)
training_dict = {'X_train': X_train, 'X_valid': X_valid,'y_train': y_train, 'y_valid': y_valid}



In [4]:
len(features)

2278

In [5]:
glove_loc = '../glove.6B.100d.txt'
glove = np.loadtxt(glove_loc,dtype='str',comments=None)

vectors = glove[:,1:].astype('float')
words = glove[:,0]

del glove

In [6]:
vocab = dict()

for sentences in features:
    sentences = clean_string(sentences)
    sentences = nltk.word_tokenize(sentences)
    for word in sentences:
        vocab[word] = 1

In [7]:
word_lookup = {word:vector for word,vector in zip(words,vectors)}

word_index = dict()
ind = 2
not_found = 0

embeds = dict()

for i,word in enumerate(vocab.keys()):
    vector = word_lookup.get(word,None)

    if vector is not None:
        word_index[word] = ind
        embeds[ind] = np.copy(vector)
        ind+=1
    else:
        not_found +=1
        word_index[word] = UNK_ID

print(f'{not_found} words not found.')

63 words not found.


In [8]:
def word_to_index(utt1):
    utt2 = [word_index.get(word,UNK_ID) for word in nltk.word_tokenize(clean_string(utt1))]
    utt3 = utt2[:MAX_LEN]
    utt4 = utt3 + [PAD_ID]*(MAX_LEN - len(utt3))
    utt5 = np.mean([embeds[i] for i in utt4 if i>1],axis=0)
    if type(utt5) == np.float64:
        utt5 = np.random.rand(100)
    if type(utt5) == np.float64:
        print('here')
    return utt5


In [9]:
training_dict['X_train'] = [word_to_index(word) for word in training_dict['X_train']]
training_dict['X_valid'] = [word_to_index(word) for word in training_dict['X_valid']]

/home/vaibhav/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/vaibhav/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import KFold

In [11]:
def svm_train(features,labels):
    clf = make_pipeline(
        StandardScaler(),
        svm.SVC(C=10.0, gamma="scale", kernel="rbf")
    )
    return clf.fit(features, labels)

In [12]:
def svm_test(clf,features,labels):
    pred = clf.predict(features)
    true = labels

    result_string = classification_report(true, pred, digits=3)
    print(confusion_matrix(true, pred))
    print(result_string)

In [13]:
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(features):
    train_x = [features[index] for index in train_index]
    train_x = [word_to_index(word) for word in train_x]
    train_y = [labels[index] for index in train_index]
    test_x = [features[index] for index in test_index]
    test_x = [word_to_index(word) for word in test_x]
    test_y = [labels[index] for index in test_index]
    clf = svm_train(train_x,train_y)
    svm_test(clf,test_x,test_y)


[[ 84  24]
 [ 13 107]]
              precision    recall  f1-score   support

         0.0      0.866     0.778     0.820       108
         1.0      0.817     0.892     0.853       120

    accuracy                          0.838       228
   macro avg      0.841     0.835     0.836       228
weighted avg      0.840     0.838     0.837       228



/home/vaibhav/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/vaibhav/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[[92 30]
 [21 85]]
              precision    recall  f1-score   support

         0.0      0.814     0.754     0.783       122
         1.0      0.739     0.802     0.769       106

    accuracy                          0.776       228
   macro avg      0.777     0.778     0.776       228
weighted avg      0.779     0.776     0.777       228



/home/vaibhav/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/vaibhav/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[[95 31]
 [12 90]]
              precision    recall  f1-score   support

         0.0      0.888     0.754     0.815       126
         1.0      0.744     0.882     0.807       102

    accuracy                          0.811       228
   macro avg      0.816     0.818     0.811       228
weighted avg      0.823     0.811     0.812       228



...

[[ 95  17]
 [ 15 101]]
              precision    recall  f1-score   support

         0.0      0.864     0.848     0.856       112
         1.0      0.856     0.871     0.863       116

    accuracy                          0.860       228
   macro avg      0.860     0.859     0.860       228
weighted avg      0.860     0.860     0.860       228



/home/vaibhav/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/vaibhav/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[[ 87   6]
 [ 29 105]]
              precision    recall  f1-score   support

         0.0      0.750     0.935     0.833        93
         1.0      0.946     0.784     0.857       134

    accuracy                          0.846       227
   macro avg      0.848     0.860     0.845       227
weighted avg      0.866     0.846     0.847       227



/home/vaibhav/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/vaibhav/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[[99 15]
 [26 87]]
              precision    recall  f1-score   support

         0.0      0.792     0.868     0.828       114
         1.0      0.853     0.770     0.809       113

    accuracy                          0.819       227
   macro avg      0.822     0.819     0.819       227
weighted avg      0.822     0.819     0.819       227



In [15]:
clf = svm_train(training_dict['X_train'],training_dict['y_train'])

svm_test(clf,training_dict['X_valid'],training_dict['y_valid'])

[[190  53]
 [ 62 195]]
              precision    recall  f1-score   support

         0.0      0.754     0.782     0.768       243
         1.0      0.786     0.759     0.772       257

    accuracy                          0.770       500
   macro avg      0.770     0.770     0.770       500
weighted avg      0.771     0.770     0.770       500

